In [1]:
from music21 import *
import tensorflow
import sys
import os
import IPython
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
import keras
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import random

2023-04-03 12:51:50.852058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 12:51:51.234940: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-03 12:51:51.234962: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-03 12:51:53.988385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [4]:
environment.set('musescoreDirectPNGPath', '../.local/bin/MuseScore-4.0.1.230121751-x86_64.AppImage')

In [17]:
filepath = "pop_songs/"

In [18]:
all_midis= []
names = []
j = 0
for i in os.listdir(filepath):
    if j > 100:
      break
    if i.endswith(".mid") and i.startswith("x"):
        tr = filepath + i
        print(tr)
        names.append(tr)
        midi = converter.parse(tr)
        all_midis.append(midi)
    j += 1
        

pop_songs/x (1).mid
pop_songs/x (37).mid
pop_songs/x (40).mid
pop_songs/x (17).mid
pop_songs/x (29).mid
pop_songs/x (41).mid
pop_songs/x (21).mid
pop_songs/x (3).mid


/home/milos/.local/lib/python3.10/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=8, channel=None, data=b'Shawn Mendes, Camila Cabello \x96 Se\xf1orita'>; getting generic Instrument
  warnings.warn(


pop_songs/x (11).mid
pop_songs/x (24).mid
pop_songs/x (23).mid
pop_songs/x (12).mid
pop_songs/x (26).mid
pop_songs/x (25).mid
pop_songs/x (16).mid
pop_songs/x (2).mid
pop_songs/x (13).mid
pop_songs/x (28).mid
pop_songs/x (45).mid
pop_songs/x (43).mid
pop_songs/x (32).mid
pop_songs/x (15).mid
pop_songs/x (47).mid
pop_songs/x (42).mid


In [19]:
def extract_notes(files):
    notes = []
    pick = None
    for f in files:
        #songs = instrument.partitionByInstrument(f)
        for part in f.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= np.array(extract_notes(all_midis))
#print("Total notes in all the Mozzart midis in the dataset:", len(Corpus))

In [20]:
Corpus = np.reshape(Corpus, (len(Corpus), 1))

In [21]:
Corpus.shape

(9323, 1)

In [22]:
Corpus

array([['4.7.11'],
       ['7.11'],
       ['0.4.7'],
       ...,
       ['F2'],
       ['G2'],
       ['G2']], dtype='<U6')

In [23]:
len(list(filter(lambda x : x == 'C3', Corpus)))

91

In [24]:
# Initialize OneHotEncoder object
encoder = OneHotEncoder()

# Fit and transform the data
one_hot_data = encoder.fit_transform(Corpus)

# Convert sparse matrix to array
one_hot_data = one_hot_data.toarray()

# Print the one-hot encoded data
print(one_hot_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [25]:
one_hot_data.shape

(9323, 135)

In [26]:
input_len = 4

In [27]:
def getXy(data, step):
    y_ind = np.arange(step, len(data), step)
    y = data[y_ind]
    rows_x = len(y)
    X = data[range(rows_x * step)]
    X = np.reshape(X, (rows_x, step, one_hot_data.shape[1]))
    return X, y

In [28]:
print(len(one_hot_data))
indeksi = np.arange(input_len, len(one_hot_data),input_len)
one_hot_data[indeksi].shape

9323


(2330, 135)

In [29]:
X, y = getXy(one_hot_data, input_len)

In [30]:
X.shape

(2330, 4, 135)

In [31]:
y.shape

(2330, 135)

In [32]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(LSTM(hidden_units, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(int(hidden_units/2)))
    model.add(Dense(units=dense_units))
    model.add(Dropout(0.1))
    model.add(Dense(units=one_hot_data.shape[1], activation='softmax'))
    model.compile(loss=tensorflow.losses.CategoricalCrossentropy(), optimizer='adam')
    return model

In [33]:
model = create_RNN(516, 164, input_shape=(input_len, one_hot_data.shape[1]), activation=['softmax', 'softmax'])

2023-04-03 13:02:48.039969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-03 13:02:48.040496: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-03 13:02:48.040625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (milos): /proc/driver/nvidia/version does not exist
2023-04-03 13:02:48.042549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 516)            1345728   
                                                                 
 dropout (Dropout)           (None, 4, 516)            0         
                                                                 
 lstm_1 (LSTM)               (None, 258)               799800    
                                                                 
 dense (Dense)               (None, 164)               42476     
                                                                 
 dropout_1 (Dropout)         (None, 164)               0         
                                                                 
 dense_1 (Dense)             (None, 135)               22275     
                                                                 
Total params: 2,210,279
Trainable params: 2,210,279
Non-

In [35]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [36]:
len(np.unique(X))

2

In [37]:
model.fit(X, y, epochs=10)

Epoch 1/10
73/73 [==============================] - 10s 67ms/step - loss: 3.9016
Epoch 2/10
73/73 [==============================] - 5s 65ms/step - loss: 2.7180
Epoch 3/10
73/73 [==============================] - 5s 66ms/step - loss: 2.3625
Epoch 4/10
73/73 [==============================] - 5s 69ms/step - loss: 2.1467
Epoch 5/10
73/73 [==============================] - 5s 65ms/step - loss: 1.9992
Epoch 6/10
73/73 [==============================] - 6s 78ms/step - loss: 1.8941
Epoch 7/10
73/73 [==============================] - 5s 67ms/step - loss: 1.7710
Epoch 8/10
73/73 [==============================] - 5s 65ms/step - loss: 1.6884
Epoch 9/10
73/73 [==============================] - 5s 70ms/step - loss: 1.6019
Epoch 10/10
73/73 [==============================] - 5s 69ms/step - loss: 1.5519


In [45]:
#izbacio dmol 2.5.9
scales = {
  "c-major": ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', '0.4.7','4.7.11', '5.9.0', '7.11.2', '9.0.4'],
  "c#-major" : ['A#3', 'C3', 'C#3', 'D#3', 'F3', 'F#3', 'G#3', '1.5.8','5.8.12', '3.6.10', '6.10.0', '8.0.3', '10.1.5'],
  "d-major": ['B3', 'C#3', 'D3', 'E3', 'F#3', 'G3', 'A3', '2.6.9','6.9.1', '4.7.11', '7.11.2', '9.1.4', '11.2.6'],
  "d#-major": ['C3', 'D3', 'D#3', 'F3', 'G3', 'G#3', 'A#3', '3.7.10','7.10.2', '5.8.0', '8.0.3', '10.2.5', '10.1.5'],
  "e-major": ['C#3', 'D#3', 'E3', 'F#3', 'G#3', 'A3', 'B3', '4.8.11','8.11.3', '6.9.1', '9.1.4', '11.3.6', '11.2.6'],
  "f-major": ['D3', 'E3', 'F3', 'G3', 'A3', 'A#3', 'C3', '5.9.0','9.0.4', '7.10.2', '10.2.5', '0.4.7', '0.3.7'],
  "f#-major": ['D#3', 'F3', 'F#3', 'G#3', 'A#3', 'B3', 'C#3', '6.10.1','10.1.5', '8.11.3', '11.3.6', '1.5.8', '1.4.8'],
  "g-major": ['E3', 'F#3', 'G3', 'A3', 'B3', 'C3', 'D3', '7.11.2','11.2.6', '9.0.4', '0.4.7', '2.6.9', '2.5.9'],
  "g#-major": ['F3', 'G3', 'G#3', 'A#3', 'C3', 'C#3', 'D#3', '8.0.3','0.3.7', '10.1.5', '1.5.8', '3.7.10', '3.6.10'],
  "a-major": ['F#3', 'G#3', 'A3', 'B3', 'C#3', 'D3', 'E3', '9.1.4','1.4.8', '11.2.6', '2.6.9', '4.8.11', '4.7.11'],
  "a#-major": ['G3', 'A3', 'A#3', 'C3', 'D3', 'D#3', 'F3', '10.2.5','2.5.9', '0.3.7', '3.7.10', '5.9.0', '5.8.0'],
  "b-major": ['G#3', 'A#3', 'B3', 'C#3', 'D#3', 'E3', 'F#3', '11.3.6','3.6.10', '1.4.8', '4.8.11', '6.10.1', '6.9.1'],
  
  "a-minor": ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', '0.4.7','4.7.11', '2.5.9', '5.9.0', '7.11.2', '9.0.4'],
  "a#-minor" : ['A#3', 'C3', 'C#3', 'D#3', 'F3', 'F#3', 'G#3', '1.5.8','5.8.12', '3.6.10', '6.10.0', '8.0.3', '10.1.5'],
  "b-minor": ['B3', 'C#3', 'D3', 'E3', 'F#3', 'G3', 'A3', '2.6.9','6.9.1', '4.7.11', '7.11.2', '9.1.4', '11.2.6'],
  "c-minor": ['C3', 'D3', 'D#3', 'F3', 'G3', 'G#3', 'A#3', '3.7.10','7.10.2', '5.8.0', '8.0.3', '10.2.5', '10.1.5'],
  "c#-minor": ['C#3', 'D#3', 'E3', 'F#3', 'G#3', 'A3', 'B3', '4.8.11','8.11.3', '6.9.1', '9.1.4', '11.3.6', '11.2.6'],
  "d-minor": ['D3', 'E3', 'F3', 'G3', 'A3', 'A#3', 'C3', '5.9.0','9.0.4', '7.10.2', '10.2.5', '0.4.7', '0.3.7'],
  "d#-minor": ['D#3', 'F3', 'F#3', 'G#3', 'A#3', 'B3', 'C#3', '6.10.1','10.1.5', '8.11.3', '11.3.6', '1.5.8', '1.4.8'],
  "e-minor": ['E3', 'F#3', 'G3', 'A3', 'B3', 'C3', 'D3', '7.11.2','11.2.6', '9.0.4', '0.4.7', '2.6.9', '2.5.9'],
  "f-minor": ['F3', 'G3', 'G#3', 'A#3', 'C3', 'C#3', 'D#3', '8.0.3','0.3.7', '10.1.5', '1.5.8', '3.7.10', '3.6.10'],
  "f#-minor": ['F#3', 'G#3', 'A3', 'B3', 'C#3', 'D3', 'E3', '9.1.4','1.4.8', '11.2.6', '2.6.9', '4.8.11', '4.7.11'],
  "g-minor": ['G3', 'A3', 'A#3', 'C3', 'D3', 'D#3', 'F3', '10.2.5','2.5.9', '0.3.7', '3.7.10', '5.9.0', '5.8.0'],
  "g#-minor": ['G#3', 'A#3', 'B3', 'C#3', 'D#3', 'E3', 'F#3', '11.3.6','3.6.10', '1.4.8', '4.8.11', '6.10.1', '6.9.1'],
}


In [47]:
input_scale = input('choose scale')

choose scalec-major


In [48]:
new_corpus = np.array([random.sample(scales[input_scale], input_len)])

In [49]:
new_corpus.shape

(1, 4)

In [50]:
new_corpus = np.reshape(new_corpus, (len(new_corpus[0]), 1))

In [51]:
new_corpus.shape

(4, 1)

In [52]:
one_hot_new_data = encoder.transform(new_corpus)
one_hot_new_data = one_hot_new_data.toarray()

In [53]:
one_hot_new_data.shape

(4, 135)

In [54]:
def funkcija(broj):
    if broj > 0.2:
        return 1
    else:
        return 0

In [55]:
s = set({})

In [56]:
1 == 1.0

True

In [57]:
def three_in_a_row(new_note, one_hot_new_data, i):
    entry = encoder.inverse_transform(new_note)
    last = encoder.inverse_transform(one_hot_new_data[i-1:i])
    before_last = encoder.inverse_transform(one_hot_new_data[i-2:i-1])
    if entry != last or entry != before_last:
        return False
    return True

In [58]:
def choose_another_note(forbidden):
    our_scale = set(scales[input_scale])
    if np.any(forbidden):
        forbidden = np.reshape(forbidden, (1, -1))
        forbidden = encoder.inverse_transform(forbidden)
        our_scale_valid = our_scale.difference({forbidden[0][0]})
    else:
        our_scale_valid = our_scale
    new_note = np.array([random.choice(list(our_scale_valid))])
    new_note = np.reshape(new_note, (len(new_note), 1))

    new_note = encoder.transform(new_note)
    new_note = new_note.toarray()
    
    return new_note
    
    

In [59]:
def in_scale(new_note):
    our_scale = set(scales[input_scale])
    if encoder.inverse_transform(new_note)[0][0] not in our_scale:
        return False
    return True

In [60]:
one_hot_new_data.shape

(4, 135)

In [61]:
i = input_len

In [62]:
while i < 112:
    new_note = model.predict(np.reshape(one_hot_new_data[i-input_len:i], (1, input_len, one_hot_data.shape[1])))
    new_note = np.array(list(map(funkcija, new_note[0])))
    new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
    
    
    while not np.any(new_note[0]) or not in_scale(new_note) or three_in_a_row(new_note, one_hot_new_data, i):
        new_note = choose_another_note(new_note[0])
        new_note = np.array(list(map(funkcija, new_note[0])))
        new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
    one_hot_new_data = np.append(one_hot_new_data, new_note, axis=0)
    i += 1



1/1 [==============================] - 0s 26ms/step


In [63]:
one_hot_new_data = encoder.inverse_transform(one_hot_new_data)

In [64]:
one_hot_new_data

array([['4.7.11'],
       ['A3'],
       ['G3'],
       ['B3'],
       ['9.0.4'],
       ['A3'],
       ['A3'],
       ['B3'],
       ['A3'],
       ['A3'],
       ['B3'],
       ['A3'],
       ['0.4.7'],
       ['B3'],
       ['D3'],
       ['D3'],
       ['4.7.11'],
       ['D3'],
       ['G3'],
       ['G3'],
       ['9.0.4'],
       ['C3'],
       ['9.0.4'],
       ['9.0.4'],
       ['B3'],
       ['A3'],
       ['0.4.7'],
       ['D3'],
       ['G3'],
       ['7.11.2'],
       ['E3'],
       ['C3'],
       ['A3'],
       ['A3'],
       ['7.11.2'],
       ['A3'],
       ['A3'],
       ['9.0.4'],
       ['A3'],
       ['A3'],
       ['4.7.11'],
       ['A3'],
       ['5.9.0'],
       ['7.11.2'],
       ['B3'],
       ['0.4.7'],
       ['7.11.2'],
       ['D3'],
       ['F3'],
       ['4.7.11'],
       ['C3'],
       ['A3'],
       ['G3'],
       ['9.0.4'],
       ['A3'],
       ['A3'],
       ['0.4.7'],
       ['A3'],
       ['A3'],
       ['9.0.4'],
       ['A3'],
       ['A3'],
  

In [65]:
one_hot_new_data.shape

(112, 1)

In [66]:
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

In [67]:
Melody = chords_n_notes(one_hot_new_data.reshape(112, ))

In [68]:
Melody_midy = stream.Stream(Melody)

In [69]:
Melody.write('midi','Melody_Generated_Better1.mid')

'Melody_Generated_Better1.mid'

In [ ]:
print("Sample Audio From Data")
IPython.display.Audio("Melody_Generated.mid") 


Sample Audio From Data


ValueError: rate must be specified when data is a numpy array or list of audio samples.